In [6]:
import os
from dotenv import load_dotenv
from utils.course import ChatOpenAI
import pandas as pd
from tqdm import tqdm

In [9]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [17]:
load_dotenv()
api_key_course = os.getenv("API_COURSE")

__Задание__: Вам дана выборка реальных текстовых заявок на жилье, которая состоит из 100 строк. В каждой заявке есть информация про число человек. Если взглянуть на эти тексты, то уже понятно, что автоматичеки вытащить число человек из заявки будет не просто. Тем не менее, это вполне реально. Попробуйте написать такой промпт с инструкцией, чтобы ваша точность была выше 80%. В этой задаче не подразумевается использование сторонних ML библиотек, кроме API ChatGPT и pandas. Хватает всего нескольких строк, чтобы решить эту задачу очень качественно.

In [91]:
# Загружаем данные
df = pd.read_csv('https://stepik.org/media/attachments/lesson/1084297/submission100lines.csv')
df.head(10)

,amount,text_id,text
0,2,14205200,Сниму жилье. 500-600 рублей сутки. Эконом клас...
1,6,319097075,ищем жилье в центре недалеко от моря с 23.07-0...
2,4,98881311,Ищем жилье на период с 18-28июля..на две семьи...
3,4,44587027,2 семьи по 3 человека (2 взрослых и ребенок) с...
4,3,352802829,Сниму недорогое жилье в лазаревском на 6 чел 3...
5,2,52157605,Ищем жилье на двоих с 7.08-14.08 . Ближе к цен...
6,6,408430986,Сниму жильё в Аше ( на улице Репина)\nС 16/29 ...
7,3,176294447,Снимем жилье с 30.06-7.07 Два взрослых и 2дете...
8,2,150296790,Снимем жильё с 30 июня по 7 июля 3 взрослых и ...
9,4,120627580,Сниму не дорогое жилье 1 взрослый и 2 ребенка ...


In [24]:
context = '\n\n##\n\n'.join(df["text"])

In [34]:
# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=api_key_course)

In [35]:
# # Напишите шаблон промпт, указав инструкцию, вопрос и выходной индикатор
# prompt_template = """Ответь на вопрос, опираясь на контекст ниже.
#
# Context: {text_input}
#
# Question: Посчитай количество человек в тексте и укажи это количество цифрой
#
# Answer: ...
# """

In [86]:
# Напишите шаблон промпт, указав инструкцию, вопрос и выходной индикатор
prompt_template = """Ответь на вопрос, опираясь только на контекст ниже.
В ответе отправьте только число, без слов.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь '0'.

Context: {text_input}

Question: Для какого количества человек требуется жилье (осторожно, не учитывай одного человека несколько раз, вчитайся в текст)?

"""

In [87]:
amount_list = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['text']):
    prompt = prompt_template.format(text_input=text_input) # Добавляем сообщение в промпт
    amount = llm.predict(prompt) # Ответ модели
    amount_list.append(amount) # Добавляем ответ в список
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 100/100 [00:52<00:00,  1.91it/s]


In [80]:
df['amount'] = amount_list # Создаём новый столбец из ответов модели
df["amount"] = df["amount"].astype(int)

In [81]:
# df.to_csv('../data/2_1_8_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)

### Проверка

In [88]:
correct_ans = pd.read_csv("../data/2_1_8_correct_ans.csv")
# solution_ans = pd.read_csv("../data/2_1_8_solution.csv")
solution_ans = df.copy()

In [89]:
data = correct_ans[["text_id","amount"]].merge(solution_ans[["text_id", "amount"]], how="left", on="text_id")
data.columns = ["text_id", "correct_ans", "predict_ans"]
data["check"] = data["correct_ans"] == data["predict_ans"]
data.pivot_table(index="check", aggfunc={"text_id":"count"})

,text_id
check,
False,5
True,95


In [ ]:
# Результаты первого запроса: 95%
# Результаты второго запроса: 95%